In [1]:
import sys
import os

sys.path.append(os.path.dirname(os.getcwd())) 

import pandas as pd
from datetime import datetime, timedelta

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
from features.load_team_stats_from_db import load_team_stats_from_db
from features.load_games_from_db import load_games_from_db
from features.load_player_stats_from_db import load_player_stats_from_db

In [3]:
df_g = load_games_from_db()
df_ts = load_team_stats_from_db()
df_ps = load_player_stats_from_db()

2025-07-01 22:23:29,034 [INFO] Successully loaded 10641 games from database


There are 10641 games for  modeling...


2025-07-01 22:23:29,263 [INFO] Successully loaded 25560 rows from database


There are 25560 team-games for  modeling...


2025-07-01 22:23:31,762 [INFO] Successully loaded 933365 rows from database


There are 933365 players for  modeling...


In [4]:
df_ps.head(3)

,game_pk,team_id,team_side,player_id,player_name,at_bats,runs_scored,hits,home_runs,rbis,walks_batting,strikeouts_batting,left_on_base,stolen_bases,innings_pitched,hits_allowed,runs_allowed,earned_runs,strikeouts_pitching,walks_pitching,pitches_thrown,putouts,assists,errors
0,634642,147,home,476595,Lucas Luetge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,634642,147,home,518934,DJ LeMahieu,4.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,0.0,None,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3.0,0.0
2,634642,147,home,650633,Michael King,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df_ps.isna().sum()

game_pk                     0
team_id                     0
team_side                   0
player_id                   0
player_name                 0
at_bats                633560
runs_scored            633560
hits                   633560
home_runs              633560
rbis                   633560
walks_batting          633560
strikeouts_batting     633560
left_on_base           633560
stolen_bases           633560
innings_pitched        817144
hits_allowed           817144
runs_allowed           817144
earned_runs            817144
strikeouts_pitching    817144
walks_pitching         817144
pitches_thrown         817144
putouts                529765
assists                529765
errors                 529765
dtype: int64

In [6]:
df_g.query("game_id==634642")

,game_id,game_date,game_date_time,home_team_id,away_team_id,home_team,away_team,home_score,away_score,state,venue,game_type
0,634642,2021-04-01,2021-04-01 17:05:00+00:00,147,141,New York Yankees,Toronto Blue Jays,2,3,Final,Yankee Stadium,R


In [7]:
df_ps.query("player_id==476595").head()

,game_pk,team_id,team_side,player_id,player_name,at_bats,runs_scored,hits,home_runs,rbis,walks_batting,strikeouts_batting,left_on_base,stolen_bases,innings_pitched,hits_allowed,runs_allowed,earned_runs,strikeouts_pitching,walks_pitching,pitches_thrown,putouts,assists,errors
0,634642,147,home,476595,Lucas Luetge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1088,634644,147,home,476595,Lucas Luetge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,0.0,15.0,0.0,0.0,0.0
1917,634607,147,home,476595,Lucas Luetge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2698,634587,147,home,476595,Lucas Luetge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3424,634599,147,home,476595,Lucas Luetge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,2.0,2.0,1.0,0.0,25.0,0.0,0.0,0.0


In [11]:
df_ps1 = df_ps.merge(df_g[['game_id','game_date_time']],
           how='left',
           left_on='game_pk',
           right_on='game_id')

# Keep games in our games dataset
df_ps2 = df_ps1[~df_ps1['game_date_time'].isna()].copy()

In [12]:
df_ps2['season'] = df_ps2['game_date_time'].dt.year

In [26]:
d2 = df_ps2[['season','game_pk','team_id','player_id','player_name','innings_pitched']].copy()

In [27]:
d2.head()

,season,game_pk,team_id,player_id,player_name,innings_pitched
0,2021,634642,147,476595,Lucas Luetge,None
1,2021,634642,147,518934,DJ LeMahieu,None
2,2021,634642,147,650633,Michael King,None
3,2021,634642,147,593334,Domingo Germán,None
4,2021,634642,147,643565,Mike Tauchman,None


In [16]:
d2.groupby('season')['player_name'].size()

season
2021    123307
2022    126278
2023    124935
2024    125803
2025     57914
Name: player_name, dtype: int64

In [17]:
d2.query("player_id==476595")['season'].value_counts(dropna=False)

season
2021    157
2022    155
2023     34
Name: count, dtype: int64

In [28]:
d2[(d2['player_id']==476595) & (d2['season']==2021)]['innings_pitched'].value_counts(dropna=False)

innings_pitched
None    101
1.0      21
1.1       9
2.0       8
0.2       6
0.1       5
1.2       4
3.0       2
4.0       1
Name: count, dtype: int64